**ETAPA 1. Descripción del Proyecto**


SQL

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

**Describe los objetivos del estudio.**

*Objetivo general*

Analizar los datos de la base de datos de libros para identificar patrones y tendencias que permitan mejorar la experiencia del usuario.

*Objetivos específicos*

- Identificar los géneros y autores más populares entre los usuarios, utilizando datos de calificaciones y reseñas.


- Desarrolar un modelo de recomendación personalizado basado en el historial de lectura y las preferencias de los usuarios.

**Cargar todas las librerías y conectar con la base de datos que se va a usar durante el proyecto.**

In [8]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Escribir consultas para datos de muestra que impriman las primeras filas**

In [9]:
# Escribir consulta para imprimir datos de muestra de todas las tablas
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    query = f"SELECT * FROM {table} LIMIT 10"
    
    display(pd.io.sql.read_sql(query, con = engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


**ETAPA 2. Análisis exploratorio**

# Encuentra el número de libros publicados después del 1 de enero de 2000.

In [10]:
# Definir la consulta SQL
sql_query = text("""
    SELECT 
        COUNT(*)
    FROM 
        books 
    WHERE publication_date > '2000-01-01'
""")

# Ejecutar la consulta y obtener el resultado
with engine.connect() as connection:
    result = connection.execute(sql_query)
    
    # Obtener el valor como un entero
    count = result.scalar()  

# Imprimir el resultado
print(f"El número de libros publicados después del 1 de enero de 2000 es: {count}")

El número de libros publicados después del 1 de enero de 2000 es: 819


# Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [11]:
# Definir la consulta SQL
sql_query = text("""
    SELECT 
        reviews.book_id, 
        COUNT(reviews.review_id) AS num_reviews, 
        AVG(ratings.rating) AS avg_rating
    FROM 
        reviews
    JOIN 
        ratings ON reviews.book_id = ratings.book_id
    GROUP BY 
        reviews.book_id
    ORDER BY 
        avg_rating DESC  
    LIMIT 10;          
""")

# Ejecutar la consulta
with engine.connect() as connection:
    result = connection.execute(sql_query)
    for row in result:
        
        # Imprimir el resultado ordenado
        print(f"Libro ID: {row.book_id}, Reseñas: {row.num_reviews}, Calificación Promedio: {row.avg_rating:.2f}")


Libro ID: 732, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 224, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 972, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 469, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 418, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 390, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 699, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 57, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 182, Reseñas: 4, Calificación Promedio: 5.00
Libro ID: 967, Reseñas: 4, Calificación Promedio: 5.00


# Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

In [12]:
# Definir la consulta SQL
sql_query = text("""
    SELECT 
        publishers.publisher, 
        COUNT(books.book_id) AS num_books
    FROM 
        books
    JOIN 
        publishers ON books.publisher_id = publishers.publisher_id
    WHERE 
        books.num_pages > 50
    GROUP BY 
        publishers.publisher
    ORDER BY 
        num_books DESC
    LIMIT 1;
""")

# Ejecutar la consulta
with engine.connect() as connection:
    result = connection.execute(sql_query)
    for row in result:
        
        # Imprimir el resultado
        print(f"Editorial: {row.publisher}, Libros publicados: {row.num_books}")

Editorial: Penguin Books, Libros publicados: 42


# Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [13]:
# Definir la consulta SQL
sql_query = text("""
    SELECT 
        a.author, 
        AVG(r.rating) AS avg_author_rating
    FROM 
        ratings r
    JOIN 
        books b ON r.book_id = b.book_id
    JOIN 
        authors a ON b.author_id = a.author_id
    GROUP BY 
        a.author
    HAVING 
        COUNT(CASE WHEN r.rating IS NOT NULL THEN r.book_id END) >= 50
    ORDER BY 
        avg_author_rating DESC
    LIMIT 1;
""")

# Ejecutar la consulta y mostrar el resultado
with engine.connect() as connection:
    result = connection.execute(sql_query)
    for row in result:
        
        # Imprimir el resultado final
        print(f"Autor: {row.author}, Calificación Promedio: {row.avg_author_rating:.2f}")

Autor: Diana Gabaldon, Calificación Promedio: 4.30


# Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [14]:
# Definir la consulta SQL
sql_query = text("""
    WITH active_users AS (
        SELECT 
            username
        FROM 
            ratings
        GROUP BY 
            username
        HAVING 
            COUNT(rating_id) > 50
    )
    SELECT 
        AVG(user_reviews.num_reviews) AS avg_text_reviews
    FROM (
        SELECT 
            rev.username, 
            COUNT(rev.review_id) AS num_reviews
        FROM 
            reviews rev
        JOIN 
            active_users au ON rev.username = au.username
        GROUP BY 
            rev.username
    ) AS user_reviews;
""")

# Ejecutar la consulta
with engine.connect() as connection:
    result = connection.execute(sql_query)
    for row in result:
        
        # Imprimir el resultado
        print(f"Número promedio de reseñas de texto: {row.avg_text_reviews:.2f}")

Número promedio de reseñas de texto: 24.33


**ETAPA 3. Describe tus conclusiones para cada una de las tareas.**

*Tarea 1: Encuentra el número de libros publicados después del 1 de enero de 2000.*
    
    
- El total de 819 libros publicados después del 1 de enero de 2000 refleja una productividad significativa en las editoriales durante las últimas décadas. Esto sugiere que el período posterior al año 2000 es un momento activo para la producción literaria.


*Tarea 2: Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.*


- El número de reseñas por libro da una idea de que tan popular es un libro entre los lectores. Los libros con muchas reseñas probablemente sean más influyentes en la comunidad.


- La calificación promedio es una medida útil para evaluar la percepción general de satisfacción de los usuarios con un libro. Los libros con calificaciones promedio altas podrían considerarse como los más destacados.


*Tarea 3: Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).*


- La editorial identificada tiene un alto volumen de publicacines de libros con más de 50 páginas, lo que indica que es una de las principales productoras de contenido literario esencial.


- Este dato refleja su posición de liderazgo dentro del mercado digital y su capacidad para enfocarse en contenido más robusto, alejándose de publicaciones menores como folletos.


*Tarea 4: Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones*


- Con una calificación promedio de 4.30, Diana Gabaldon se posiciona como una de las autoras más valoradas entre los lectores. Este resultado sugiere que sus libros tienen una gran aceptación y son consistentemente bien recibidos.


- La alta calificación refleja una percepción positiva y coherente por parte de los lectores, lo que podría deberse a su estilo de escritura, narrativa envolvente o los temas que aborda en sus obras.


*Tarea 5: Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.*


- Este grupo de usuarios representa una parte importante de los lectores activos en la plataforma. Fomentar y reconocer su participación podría ser estratégico para aumentar la interacción y la creación de contenido de calidad. 